# LlamaIndexでWorkflows

## 目次
- [概要](#概要)
- [参考](#参考)
- [チェック](#チェック)
- [準備](#準備)
- [最も簡単なエージェント](#最も簡単なエージェント)
- [エージェントにRAGを追加する](#エージェントにRAGを追加する)

## 概要
- LlamaIndex（公式）をトレースして基本的な利用方法を確認する。
- 破壊的に変更が発生するまで使えるでしょう。
- 破壊的に変更が発生後は、公式サイトの当該バージョンの情報（≒一次情報）をあたって。

## 参考

LlamaIndex - .NET 開発基盤部会 Wiki  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?LlamaIndex

## チェック

In [1]:
#!pip list

In [2]:
#%env

## 準備

### インストレーション

```Python
!pip install llama-index-utils-workflow
```

### ライブラリ読み込み

In [3]:
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    StartEvent,
    StopEvent,
    Context,
)
from llama_index.utils.workflow import (
    draw_all_possible_flows,
    draw_most_recent_execution,
)

### 関数定義
HTML表示

In [4]:
from IPython.display import display, IFrame #HTML

# IFrameでないと上手く表示できないがGitHub上で表示されない。
def display_html(file_path = "example.html"):
    #with open(file_path, 'r', encoding='utf-8') as file:
    #    html_content = file.read()
    #print(html_content)
    display(IFrame(file_path, width="100%", height="300"))

### LLMの設定

#### OpenAI

In [5]:
from llama_index.llms.openai import OpenAI
#llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

#### Ollama

In [6]:
from llama_index.llms.ollama import Ollama
#llm = Ollama(model="Llama3", temperature=0, request_timeout=360.0)

## シングルステップ

### ワークフロー定義

In [7]:
class MyWorkflow(Workflow):
    @step
    async def my_step(self, ev: StartEvent) -> StopEvent:
        # do something here
        return StopEvent(result="Hello, world!")

### ワークフロー実行

In [8]:
w = MyWorkflow(timeout=10, verbose=True)
result = await w.run()
print(result)

Running step my_step
Step my_step produced event StopEvent
Hello, world!


## ループと分岐

### ループ

#### カスタム・イベント

In [9]:
class LoopEvent(Event):
    loop_output: str
class FirstEvent(Event):
    first_output: str

#### ワークフロー定義

In [10]:
import random

class LoopWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent | LoopEvent) -> FirstEvent | LoopEvent:
        if random.randint(0, 1) == 0:
            print("Bad thing happened")
            return LoopEvent(loop_output="Back to step one.")
        else:
            print("Good thing happened")
            return FirstEvent(first_output="First step complete.")

    @step
    async def first_step(self, ev: FirstEvent) -> StopEvent:
        # do something here
        return StopEvent(result="Hello, world!")

#### ワークフロー定義の表示

In [11]:
filename="./llamaindex/LoopWorkflow.html"
draw_all_possible_flows(LoopWorkflow, filename)
display_html(filename)

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.FirstEvent'>
<class '__main__.LoopEvent'>
./llamaindex/LoopWorkflow.html


#### ワークフロー実行

In [12]:
w = LoopWorkflow(verbose=True)
result = await w.run()
print(result)

Running step step_one
Good thing happened
Step step_one produced event FirstEvent
Running step first_step
Step first_step produced event StopEvent
Hello, world!


#### ワークフロー結果の表示

In [13]:
filename="./llamaindex/LoopWorkflowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/LoopWorkflowExec.html


### 分岐

#### カスタム・イベント

In [14]:
class BranchA1Event(Event):
    payload: str
class BranchA2Event(Event):
    payload: str
class BranchB1Event(Event):
    payload: str
class BranchB2Event(Event):
    payload: str

#### ワークフロー定義

In [15]:
import random

class BranchWorkflow(Workflow):
    @step
    async def start(self, ev: StartEvent) -> BranchA1Event | BranchB1Event:
        if random.randint(0, 1) == 0:
            print("Go to branch A")
            return BranchA1Event(payload="Branch A")
        else:
            print("Go to branch B")
            return BranchB1Event(payload="Branch B")

    @step
    async def step_a1(self, ev: BranchA1Event) -> BranchA2Event:
        print(ev.payload)
        return BranchA2Event(payload=ev.payload)

    @step
    async def step_b1(self, ev: BranchB1Event) -> BranchB2Event:
        print(ev.payload)
        return BranchB2Event(payload=ev.payload)

    @step
    async def step_a2(self, ev: BranchA2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch A complete.")

    @step
    async def step_b2(self, ev: BranchB2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch B complete.")

#### ワークフロー定義の表示

In [16]:
filename="./llamaindex/BranchWorkflow.html"
draw_all_possible_flows(BranchWorkflow, filename)
display_html(filename)

<class 'NoneType'>
<class '__main__.BranchA1Event'>
<class '__main__.BranchB1Event'>
<class '__main__.BranchA2Event'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.BranchB2Event'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/BranchWorkflow.html


#### ワークフロー実行

In [17]:
w = BranchWorkflow(verbose=True)
result = await w.run()
print(result)

Running step start
Go to branch A
Step start produced event BranchA1Event
Running step step_a1
Branch A
Step step_a1 produced event BranchA2Event
Running step step_a2
Branch A
Step step_a2 produced event StopEvent
Branch A complete.


#### ワークフロー結果の表示

In [18]:
filename="./llamaindex/BranchWorkflowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/BranchWorkflowExec.html


## 状態

### カスタム・イベント

In [19]:
class SetupEvent(Event):
    query: str
class StepTwoEvent(Event):
    query: str

### ワークフロー定義

In [20]:
class StatefulFlow(Workflow):
    @step
    async def setup(self, ctx: Context, ev: SetupEvent) -> StartEvent:
        # load data
        await ctx.set("some_database", [1, 2, 3])
        return StartEvent(query=ev.query)

    @step
    async def start(self, ctx: Context, ev: StartEvent) -> SetupEvent | StepTwoEvent:
        db = await ctx.get("some_database", default=None)
        if db is None:
            print("Need to load data")
            return SetupEvent(query=ev.query)

        # do something with the query
        return StepTwoEvent(query=ev.query)

    @step
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StopEvent:
        # do something with the data
        print("Data is ", await ctx.get("some_database"))
    
        return StopEvent(result=await ctx.get("some_database"))

### ワークフロー表示

In [21]:
filename="./llamaindex/StatefulFlow.html"
draw_all_possible_flows(StatefulFlow, filename)
display_html(filename)

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StartEvent'>
<class '__main__.SetupEvent'>
<class '__main__.StepTwoEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/StatefulFlow.html


### ワークフロー実行

In [22]:
w = StatefulFlow(timeout=10, verbose=True)
result = await w.run(query="Some query")
print(result)

Running step start
Need to load data
Step start produced event SetupEvent
Running step setup
Step setup produced event StartEvent
Running step start
Step start produced event StepTwoEvent
Running step step_two
Data is  [1, 2, 3]
Step step_two produced event StopEvent
[1, 2, 3]


### ワークフロー結果の表示

In [23]:
filename="./llamaindex/StatefulFlowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/StatefulFlowExec.html


## ストリーミングイベント

In [24]:
import asyncio

### カスタム・イベント

In [25]:
class FirstEvent(Event):
    first_output: str
class SecondEvent(Event):
    second_output: str
    response: str
class ProgressEvent(Event):
    msg: str

### ワークフロー定義

In [26]:
class StreamingWorkflow(Workflow):
    @step
    async def step_one(self, ctx: Context, ev: StartEvent) -> FirstEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step one is happening"))
        return FirstEvent(first_output="First step complete.")

    @step
    async def step_two(self, ctx: Context, ev: FirstEvent) -> SecondEvent:
        llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
        generator = await llm.astream_complete(
            "Please give me the first 3 paragraphs of Moby Dick, a book in the public domain."
        )
        async for response in generator:
            # Allow the workflow to stream this piece of response
            ctx.write_event_to_stream(ProgressEvent(msg=response.delta))
        return SecondEvent(
            second_output="Second step complete, full response attached",
            response=str(response),
        )

    @step
    async def step_three(self, ctx: Context, ev: SecondEvent) -> StopEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step three is happening"))
        return StopEvent(result="Workflow complete.")

### ワークフロー表示

In [27]:
filename="./llamaindex/StreamingWorkflow.html"
draw_all_possible_flows(StatefulFlow, filename)
display_html(filename)

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StartEvent'>
<class '__main__.SetupEvent'>
<class '__main__.StepTwoEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/StreamingWorkflow.html


### ワークフロー実行

#### 非同期関数を定義

In [28]:
async def main():
    w = StreamingWorkflow(timeout=30, verbose=True)
    handler = w.run(first_input="Start the workflow.")

    async for ev in handler.stream_events():
        if isinstance(ev, ProgressEvent):
            print(ev.msg)

    final_result = await handler
    print("Final result", final_result)

#### Pythonのバージョン
次第で非同期の実行方法方が違うらしい。

In [29]:
!python3 -V

Python 3.10.12


In [30]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.29.0
ipykernel        : 6.29.5
ipywidgets       : not installed
jupyter_client   : 8.6.3
jupyter_core     : 5.7.2
jupyter_server   : 2.14.2
jupyterlab       : 4.3.1
nbclient         : 0.10.0
nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : not installed
qtconsole        : not installed
traitlets        : 5.14.3


#### 非同期関数の実行

In [31]:
#if __name__ == "__main__":
#    asyncio.run(main())
result = await main()

Running step step_one
Step step_one produced event FirstEvent
Running step step_two
Step one is happening

Chapter
 
1
:
 L
oom
ings



Call
 me
 Ish
ma
el
.
 Some
 years
 ago
—
never
 mind
 how
 long
 precisely
—
having
 little
 or
 no
 money
 in
 my
 purse
,
 and
 nothing
 particular
 to
 interest
 me
 on
 shore
,
 I
 thought
 I
 would
 sail
 about
 a
 little
 and
 see
 the
 wat
ery
 part
 of
 the
 world
.
 It
 is
 a
 way
 I
 have
 of
 driving
 off
 the
 sple
en
 and
 regulating
 the
 circulation
.
 Whenever
 I
 find
 myself
 growing
 grim
 about
 the
 mouth
;
 whenever
 it
 is
 a
 damp
,
 dr
izzly
 November
 in
 my
 soul
;
 whenever
 I
 find
 myself
 invol
unt
arily
 pa
using
 before
 coffin
 warehouses
,
 and
 bringing
 up
 the
 rear
 of
 every
 funeral
 I
 meet
;
 and
 especially
 whenever
 my
 hy
pos
 get
 such
 an
 upper
 hand
 of
 me
,
 that
 it
 requires
 a
 strong
 moral
 principle
 to
 prevent
 me
 from
 deliberately
 stepping
 into
 the
 street
,
 and
 method
ically
 knocki

### ワークフロー結果の表示

In [32]:
filename="./llamaindex/StreamingWorkflowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/StreamingWorkflowExec.html


## 同時実行

### ParallelFlow

#### ワークフロー定義

In [33]:
class ParallelFlow(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> StepTwoEvent:
        ctx.send_event(StepTwoEvent(query="Query 1"))
        ctx.send_event(StepTwoEvent(query="Query 2"))
        ctx.send_event(StepTwoEvent(query="Query 3"))

    @step(num_workers=4)
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StopEvent:
        print("Running slow query ", ev.query)
        await asyncio.sleep(random.randint(1, 5))

        return StopEvent(result=ev.query)

#### ワークフロー表示

In [34]:
filename="./llamaindex/ParallelFlow.html"
draw_all_possible_flows(ParallelFlow, filename)
display_html(filename)

<class 'NoneType'>
<class '__main__.StepTwoEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/ParallelFlow.html


#### ワークフロー実行

In [35]:
w = ParallelFlow(timeout=10, verbose=True)
result = await w.run()
print(result)

Running step start
Step start produced no event
Running step step_two
Running slow query  Query 1
Running step step_two
Running slow query  Query 2
Running step step_two
Running slow query  Query 3
Step step_two produced event StopEvent
Query 1


#### ワークフロー結果の表示

In [36]:
filename="./llamaindex/ParallelFlowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/ParallelFlowExec.html


### ConcurrentFlow1

#### カスタム・イベント

In [37]:
class StepThreeEvent(Event):
    result: str

#### ワークフロー定義

In [38]:
class ConcurrentFlow1(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> StepTwoEvent:
        ctx.send_event(StepTwoEvent(query="Query 1"))
        ctx.send_event(StepTwoEvent(query="Query 2"))
        ctx.send_event(StepTwoEvent(query="Query 3"))

    @step(num_workers=4)
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StepThreeEvent:
        print("Running query ", ev.query)
        await asyncio.sleep(random.randint(1, 5))
        return StepThreeEvent(result=ev.query)

    @step
    async def step_three(self, ctx: Context, ev: StepThreeEvent) -> StopEvent:
        # wait until we receive 3 events
        result = ctx.collect_events(ev, [StepThreeEvent] * 3)
        if result is None:
            return None

        # do something with all 3 results together
        print(result)
        return StopEvent(result="Done")

#### ワークフロー表示

In [39]:
filename="./llamaindex/ConcurrentFlow1.html"
draw_all_possible_flows(ConcurrentFlow1, filename)
display_html(filename)

<class 'NoneType'>
<class '__main__.StepTwoEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.StepThreeEvent'>
./llamaindex/ConcurrentFlow1.html


#### ワークフロー実行

In [40]:
w = ConcurrentFlow1(timeout=10, verbose=True)
result = await w.run()
print(result)

Running step start
Step start produced no event
Running step step_two
Running query  Query 1
Running step step_two
Running query  Query 2
Running step step_two
Running query  Query 3
Step step_two produced event StepThreeEvent
Step step_two produced event StepThreeEvent
Step step_two produced event StepThreeEvent
Running step step_three
Step step_three produced no event
Running step step_three
Step step_three produced no event
Running step step_three
[StepThreeEvent(result='Query 1'), StepThreeEvent(result='Query 2'), StepThreeEvent(result='Query 3')]
Step step_three produced event StopEvent
Done


#### ワークフロー結果の表示

In [41]:
filename="./llamaindex/ConcurrentFlow1Exec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/ConcurrentFlow1Exec.html


### ConcurrentFlow2

#### カスタム・イベント

In [42]:
class StepAEvent(Event):
    query: str
class StepBEvent(Event):
    query: str
class StepCEvent(Event):
    query: str

class StepACompleteEvent(Event):
    result: str
class StepBCompleteEvent(Event):
    result: str
class StepCCompleteEvent(Event):
    result: str

#### ワークフロー定義

In [43]:
class ConcurrentFlow2(Workflow):
    @step
    async def start(
        self, ctx: Context, ev: StartEvent
    ) -> StepAEvent | StepBEvent | StepCEvent:
        ctx.send_event(StepAEvent(query="Query 1"))
        ctx.send_event(StepBEvent(query="Query 2"))
        ctx.send_event(StepCEvent(query="Query 3"))

    @step
    async def step_a(self, ctx: Context, ev: StepAEvent) -> StepACompleteEvent:
        print("Doing something A-ish")
        return StepACompleteEvent(result=ev.query)

    @step
    async def step_b(self, ctx: Context, ev: StepBEvent) -> StepBCompleteEvent:
        print("Doing something B-ish")
        return StepBCompleteEvent(result=ev.query)

    @step
    async def step_c(self, ctx: Context, ev: StepCEvent) -> StepCCompleteEvent:
        print("Doing something C-ish")
        return StepCCompleteEvent(result=ev.query)

    @step
    async def step_three(
        self,
        ctx: Context,
        ev: StepACompleteEvent | StepBCompleteEvent | StepCCompleteEvent,
    ) -> StopEvent:
        print("Received event ", ev.result)

        # wait until we receive 3 events
        if (
            ctx.collect_events(
                ev,
                [StepACompleteEvent, StepBCompleteEvent, StepCCompleteEvent], 
            )
            is None
        ):
            return None

        # do something with all 3 results together
        return StopEvent(result="Done")

#### ワークフロー表示

In [44]:
filename="./llamaindex/ConcurrentFlow2.html"
draw_all_possible_flows(ConcurrentFlow2, filename)
display_html(filename)

<class 'NoneType'>
<class '__main__.StepAEvent'>
<class '__main__.StepBEvent'>
<class '__main__.StepCEvent'>
<class '__main__.StepACompleteEvent'>
<class '__main__.StepBCompleteEvent'>
<class '__main__.StepCCompleteEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/ConcurrentFlow2.html


#### ワークフロー実行

In [45]:
w = ConcurrentFlow2(timeout=10, verbose=True)
result = await w.run()
print(result)

Running step start
Step start produced no event
Running step step_a
Doing something A-ish
Step step_a produced event StepACompleteEvent
Running step step_b
Doing something B-ish
Step step_b produced event StepBCompleteEvent
Running step step_c
Doing something C-ish
Step step_c produced event StepCCompleteEvent
Running step step_three
Received event  Query 1
Step step_three produced no event
Running step step_three
Received event  Query 2
Step step_three produced no event
Running step step_three
Received event  Query 3
Step step_three produced event StopEvent
Done


#### ワークフロー結果の表示

In [46]:
filename="./llamaindex/ConcurrentFlow2Exec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/ConcurrentFlow2Exec.html


## サブクラス化

### メイン

#### カスタム・イベント

In [47]:
class Step2Event(Event):
    query: str
class Step3Event(Event):
    query: str

#### ワークフロー定義

In [48]:
class MainWorkflow(Workflow):
    @step
    async def start(self, ev: StartEvent) -> Step2Event:
        print("Starting up")
        return Step2Event(query=ev.query)

    @step
    async def step_two(self, ev: Step2Event) -> Step3Event:
        print("Sending an email")
        return Step3Event(query=ev.query)

    @step
    async def step_three(self, ev: Step3Event) -> StopEvent:
        print("Finishing up")
        return StopEvent(result=ev.query)

#### ワークフロー表示

In [49]:
filename="./llamaindex/MainWorkflow.html"
draw_all_possible_flows(MainWorkflow, filename)
display_html(filename)

<class 'NoneType'>
<class '__main__.Step2Event'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.Step3Event'>
./llamaindex/MainWorkflow.html


#### ワークフロー実行

In [50]:
w = MainWorkflow(timeout=10, verbose=True)
result = await w.run(query="Initial query")
print(result)

Running step start
Starting up
Step start produced event Step2Event
Running step step_two
Sending an email
Step step_two produced event Step3Event
Running step step_three
Finishing up
Step step_three produced event StopEvent
Initial query


#### ワークフロー結果の表示

In [51]:
filename="./llamaindex/MainWorkflowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/MainWorkflowExec.html


### サブ

#### カスタム・イベント

In [52]:
class Step2BEvent(Event):
    query: str

#### ワークフロー定義

In [53]:
class SubclassingWorkflow(MainWorkflow):
    @step
    async def step_two(self, ev: Step2Event) -> Step2BEvent:
        print("Sending an email (override?)")
        return Step2BEvent(query=ev.query)

    @step
    async def step_two_b(self, ev: Step2BEvent) -> Step3Event:
        print("Also sending a text message")
        return Step3Event(query=ev.query)


#### ワークフロー表示

In [54]:
filename="./llamaindex/SubclassingWorkflow.html"
draw_all_possible_flows(SubclassingWorkflow, filename)
display_html(filename)

<class 'NoneType'>
<class '__main__.Step2Event'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.Step2BEvent'>
<class '__main__.Step3Event'>
./llamaindex/SubclassingWorkflow.html


#### ワークフロー実行

In [55]:
w = SubclassingWorkflow(timeout=10, verbose=True)
result = await w.run(query="Initial query")
print(result)

Running step start
Starting up
Step start produced event Step2Event
Running step step_two
Sending an email (override?)
Step step_two produced event Step2BEvent
Running step step_two_b
Also sending a text message
Step step_two_b produced event Step3Event
Running step step_three
Finishing up
Step step_three produced event StopEvent
Initial query


#### ワークフロー結果の表示

In [56]:
filename="./llamaindex/SubclassingWorkflowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/SubclassingWorkflowExec.html


## ネスト

### カスタム・イベント

In [57]:
class Step2Event(Event):
    query: str

### ワークフロー定義

#### DefaultSubflow

In [58]:
class DefaultSubflow(Workflow):
    @step()
    async def sub_start(self, ctx: Context, ev: StartEvent) -> StopEvent:
        print("Doing basic reflection")
        return StopEvent(result="Improved query")

#### MainWorkflow

In [59]:
class MainWorkflow(Workflow):
    @step
    async def start(
        self,
        ctx: Context,
        ev: StartEvent,
        reflection_workflow: Workflow = DefaultSubflow(),
    ) -> Step2Event:
        print("Need to run reflection")
        res = await reflection_workflow.run(query=ev.query)
        return Step2Event(query=res)

    @step
    async def step_two(self, ctx: Context, ev: Step2Event) -> StopEvent:
        print("Query is ", ev.query)
        # do something with the query here
        return StopEvent(result=ev.query)

#### ReflectionFlow

In [60]:
class ReflectionFlow(Workflow):
    @step
    async def sub_start(self, ctx: Context, ev: StartEvent) -> StopEvent:
        print("Doing custom reflection")
        return StopEvent(result="Improved query")

### ワークフロー表示

In [61]:
filename="./llamaindex/MainWorkflow.html"
draw_all_possible_flows(MainWorkflow, filename)
display_html(filename)

<class 'NoneType'>
<class '__main__.Step2Event'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/MainWorkflow.html


### ワークフロー実行

#### DefaultSubflow

In [62]:
w = MainWorkflow(timeout=10, verbose=True)
result = await w.run(query="Initial query")
print(result)

Running step start
Need to run reflection
Doing basic reflection
Step start produced event Step2Event
Running step step_two
Query is  Improved query
Step step_two produced event StopEvent
Improved query


#### ReflectionFlow

In [63]:
w = MainWorkflow(timeout=10, verbose=True)
w.add_workflows(reflection_workflow=ReflectionFlow())
result = await w.run(query="Initial query")
print(result)

Running step start
Need to run reflection
Doing custom reflection
Step start produced event Step2Event
Running step step_two
Query is  Improved query
Step step_two produced event StopEvent
Improved query


### ワークフロー結果の表示

In [64]:
filename="./llamaindex/MainWorkflowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/MainWorkflowExec.html


## 非制限構文

### ワークフロー定義

In [65]:
class TestWorkflow(Workflow):
    pass

@step(workflow=TestWorkflow)
def some_step(ev: StartEvent) -> StopEvent:
    return StopEvent()

### ワークフロー表示

In [66]:
filename="./llamaindex/TestWorkflow.html"
draw_all_possible_flows(TestWorkflow, filename)
display_html(filename)

<class 'NoneType'>
./llamaindex/TestWorkflow.html


### ワークフロー実行

In [67]:
w = TestWorkflow(timeout=10, verbose=True)
result = await w.run(query="Some query")
print(result)

Running step some_step
Step some_step produced event StopEvent
None


### ワークフロー結果の表示

In [68]:
filename="./llamaindex/TestWorkflowExec.html"
draw_most_recent_execution(w, filename)
display_html(filename)

./llamaindex/TestWorkflowExec.html


## 可観測性

### 視覚化

#### 定義の視覚化
既出の、`draw_all_possible_flows`を参照。

#### 結果の視覚化
既出の、`draw_most_recent_execution`を参照。

### 詳細モード
既出の、`verbose=True`を参照。

### 段階的な実行
エラーになる。

```Python
w = ConcurrentFlow2(timeout=10, verbose=True)
handler = w.run()

while not handler.is_done():
    # run_step returns the step's output event
    # run_stepはステップの出力イベントを返す
    ev = await handler.run_step()
    # can make modifications to the results before dispatching the event
    # イベントをディスパッチする前に結果を修正できる
    #val = ev.get("some_key")
    #ev.set("some_key", new_val)
    # can also inspect context
    # ctxも検査できる
    #val = await handler.ctx.get("key")
    handler.ctx.send_event(ev)
    continue

# get the result
result = handler.result()
```

### チェックポイント
エラーになる。

```Python
from llama_index.core.workflow.checkpointer import WorkflowCheckpointer

w = ConcurrentFlow()
w_ckptr = WorkflowCheckpointer(workflow=w)

# run the workflow via w_ckptr to get checkpoints
handler = w_cptr.run()
await handler

# view checkpoints of the last run
w_ckptr.checkpoints[handler.run_id]

# run from a previous ckpt
ckpt = w_ckptr.checkpoints[handler.run_id][0]
handler = w_ckptr.run_from(checkpoint=ckpt)
await handler
```